##### Copyright 2021 Les auteurs TF-Agents.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Pilotes lhn jhh

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Voir sur TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/4_drivers_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Exécuter dans Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/4_drivers_tutorial.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Voir la source sur GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/4_drivers_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Télécharger le cahier</a></td>
</table>

## Introduction

Un modèle courant dans l'apprentissage par renforcement consiste à exécuter une politique dans un environnement pendant un nombre spécifié d'étapes ou d'épisodes. Cela se produit, par exemple, lors de la collecte de données, de l'évaluation et de la génération d'une vidéo de l'agent.

While this is relatively straightforward to write in python, it is much more complex to write and debug in TensorFlow because it involves `tf.while` loops, `tf.cond` and `tf.control_dependencies`. Therefore we abstract this notion of a run loop into a class called `driver`, and provide well tested implementations both in Python and TensorFlow.

De plus, les données rencontrées par le pilote à chaque étape sont enregistrées dans un tuple nommé appelé Trajectory et diffusées à un ensemble d'observateurs tels que des tampons de relecture et des métriques. Ces données incluent l'observation de l'environnement, l'action recommandée par la politique, la récompense obtenue, le type de l'étape actuelle et suivante, etc.

## Installation

# Ensemble

Si vous n'avez pas encore installé tf-agents ou gym, lancez :

In [ ]:
!pip install tf-agents


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.policies import random_py_policy
from tf_agents.policies import random_tf_policy
from tf_agents.metrics import py_metrics
from tf_agents.metrics import tf_metrics
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_episode_driver

## Python Drivers

La classe `PyDriver` prend un environnement python, une politique python et une liste d'observateurs à mettre à jour à chaque étape. La méthode principale est `run()` , qui échelonne l'environnement à l'aide des actions de la stratégie jusqu'à ce qu'au moins l'un des critères d'arrêt suivants soit rempli : le nombre d'étapes atteint `max_steps` ou le nombre d'épisodes atteint `max_episodes` .

La mise en œuvre est grossièrement la suivante :

```python
class PyDriver(object):

  def __init__(self, env, policy, observers, max_steps=1, max_episodes=1):
    self._env = env
    self._policy = policy
    self._observers = observers or []
    self._max_steps = max_steps or np.inf
    self._max_episodes = max_episodes or np.inf

  def run(self, time_step, policy_state=()):
    num_steps = 0
    num_episodes = 0
    while num_steps < self._max_steps and num_episodes < self._max_episodes:

      # Compute an action using the policy for the given time_step
      action_step = self._policy.action(time_step, policy_state)

      # Apply the action to the environment and get the next step
      next_time_step = self._env.step(action_step.action)

      # Package information into a trajectory
      traj = trajectory.Trajectory(
         time_step.step_type,
         time_step.observation,
         action_step.action,
         action_step.info,
         next_time_step.step_type,
         next_time_step.reward,
         next_time_step.discount)

      for observer in self._observers:
        observer(traj)

      # Update statistics to check termination
      num_episodes += np.sum(traj.is_last())
      num_steps += np.sum(~traj.is_boundary())

      time_step = next_time_step
      policy_state = action_step.state

    return time_step, policy_state

```

Passons maintenant à l'exemple de l'exécution d'une politique aléatoire sur l'environnement CartPole, en enregistrant les résultats dans un tampon de relecture et en calculant certaines métriques.

In [ ]:
env = suite_gym.load('CartPole-v0')
policy = random_py_policy.RandomPyPolicy(time_step_spec=env.time_step_spec(), 
                                         action_spec=env.action_spec())
replay_buffer = []
metric = py_metrics.AverageReturnMetric()
observers = [replay_buffer.append, metric]
driver = py_driver.PyDriver(
    env, policy, observers, max_steps=20, max_episodes=1)

initial_time_step = env.reset()
final_time_step, _ = driver.run(initial_time_step)

print('Replay Buffer:')
for traj in replay_buffer:
  print(traj)

print('Average Return: ', metric.result())

## Pilotes TensorFlow

Nous avons également des pilotes dans TensorFlow qui sont fonctionnellement similaires aux pilotes Python, mais utilisent des environnements TF, des politiques TF, des observateurs TF, etc. Nous avons actuellement 2 pilotes TensorFlow : `DynamicStepDriver` , qui se termine après un nombre donné d'étapes d'environnement (valides) et `DynamicEpisodeDriver` , qui se termine après un certain nombre d'épisodes. Regardons un exemple du DynamicEpisode en action.


In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

tf_policy = random_tf_policy.RandomTFPolicy(action_spec=tf_env.action_spec(),
                                            time_step_spec=tf_env.time_step_spec())


num_episodes = tf_metrics.NumberOfEpisodes()
env_steps = tf_metrics.EnvironmentSteps()
observers = [num_episodes, env_steps]
driver = dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env, tf_policy, observers, num_episodes=2)

# Initial driver.run will reset the environment and initialize the policy.
final_time_step, policy_state = driver.run()

print('final_time_step', final_time_step)
print('Number of Steps: ', env_steps.result().numpy())
print('Number of Episodes: ', num_episodes.result().numpy())

In [ ]:
# Continue running from previous state
final_time_step, _ = driver.run(final_time_step, policy_state)

print('final_time_step', final_time_step)
print('Number of Steps: ', env_steps.result().numpy())
print('Number of Episodes: ', num_episodes.result().numpy())